In [1]:
import pandas as pd
import pandasql as ps

In [2]:
filename = '../data/exchange_metrics.csv'
df_exchanges = pd.read_csv(filename)

In [3]:
df_exchanges.iloc[0]

coin_id                                                                    cosmos
coin_name                                                                  Cosmos
symbol                                                                       atom
base                            IBC/27394FB092D2ECCD56123C74F36E4C1F926001CEAD...
target                                                                      UOSMO
market_name                                                               Osmosis
market_identifier                                                         osmosis
market_has_trading_incentive                                                False
price                                                                    4.753753
volume                                                                 40575886.0
pp_total_volume                                                          0.024948
is_stale                                                                    False
Name: 0, dtype: 

In [4]:
filename = '../data/coin_metrics.csv'
df_coins = pd.read_csv(filename)

In [5]:
df_coins.iloc[0]

coin_id                                                 cosmos
coin_name                                               Cosmos
symbol                                                    atom
price                                                    39.42
volume_24h                                        1626408552.0
market_cap                                       11151802465.0
fully_diluted_valuation                                      0
circulating_supply                            285438514.681725
total_supply                                               NaN
max_supply                                                 NaN
twitter_followers                                       293282
facebook_likes                                             NaN
reddit_average_posts_48h                                   5.2
reddit_average_comments_48h                              148.3
reddit_subscribers                                       52048
reddit_accounts_active_48h                             

### Exchange Metrics

In [18]:
sql_string = """
    WITH num_exchanges AS (
        SELECT coin_name, symbol, COUNT(DISTINCT market_name) AS num_exchanges
        FROM df_exchanges
        GROUP BY 1, 2
    ),
    
    top_exchange_by_volume AS (
        SELECT A.coin_name, A.symbol, A.market_name, B.max_volume
        FROM df_exchanges AS A
        INNER JOIN (
            SELECT coin_name, symbol, MAX(volume) AS max_volume
            FROM df_exchanges
            GROUP BY 1, 2
        ) AS B
        ON 
            A.coin_name = B.coin_name AND
            A.symbol = B.symbol AND 
            A.volume = B.max_volume
        GROUP by 1, 2, 3
    )    
    
    SELECT 
        DISTINCT
        A.coin_name, 
        A.symbol, 
        A.price, 
        B.num_exchanges,
        C.market_name,
        max_volume AS market_volume,
        volume_24h,
        max_volume*1.0/ volume_24h*1.0 AS pp_volume
    FROM df_coins AS A 
    JOIN num_exchanges AS B
    ON 
        A.coin_name = B.coin_name AND 
        A.symbol = B.symbol 
    JOIN top_exchange_by_volume AS C
    ON 
        A.coin_name = C.coin_name AND 
        A.symbol = C.symbol
    ORDER BY volume_24h DESC
"""

df_final = ps.sqldf(sql_string)

In [19]:
df_final.head(20)

,coin_name,symbol,price,num_exchanges,market_name,market_volume,volume_24h,pp_volume
0,Cosmos,atom,39.420000,47,Binance,289245902.0,1.626409e+09,0.177843
1,Crypto.com Coin,cro,0.536105,42,Crypto.com Exchange,40498966.0,1.831117e+08,0.221171
2,Band Protocol,band,6.820000,57,Binance,25921868.0,1.142012e+08,0.226984
3,Osmosis,osmo,8.180000,3,Osmosis,40575886.0,9.711715e+07,0.417804
4,Kava,kava,4.760000,30,Binance,35734970.0,8.937829e+07,0.399817
5,Fetch.ai,fet,0.604789,29,Binance,28603449.0,7.312365e+07,0.391165
6,ICON,icx,1.200000,33,Binance,12646689.0,5.801507e+07,0.217990
7,Quant,qnt,180.270000,30,FatBTC,6386215.0,3.616361e+07,0.176592
8,Whitecoin,xwc,0.813426,7,ZB,2750415.0,1.430826e+07,0.192226
9,CertiK,ctk,1.720000,13,Binance,4028003.0,7.344307e+06,0.548452
